<a href="https://colab.research.google.com/github/tommcrojo/hybrid_nn_advisor/blob/main/alu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

###### start

In [1]:
!pip install polars xlsx2csv python-calamine fastexcel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.0/886.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.5 MB/s eta 0:00:00


In [2]:
#!pip install uv
#!uv pip install polars xlsx2csv python-calamine fastexcel

In [3]:
import re
import os
import datetime
from google.colab import drive
from google.colab import files
import io
from google.auth import default
import gspread
from google.auth.transport.requests import Request
from google.colab import auth
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import polars as pl
file_path = '/content/drive/MyDrive/BBDDAlumnos/bbdd_alumnos_raw_2025-04.tsv'
raw_o = pl.read_csv(file_path, separator='\t', has_header=False)

raw_esi = pl.read_excel(
    source='/content/drive/MyDrive/BBDDAlumnos/ESI alumnado.xlsm',
    sheet_id=1,
    engine='calamine',
)

raw_tp = pl.read_excel(
    source='/content/drive/MyDrive/BBDDAlumnos/TP alumnado.xlsx',
    sheet_name="L0",
    engine='calamine',
)


##### Dataprep titOficial

In [5]:
# Get the second row as a list
new_headers = raw_o.row(1)

# Remove the first two rows
raw_o = raw_o.slice(2)

# Rename the columns
raw_o = raw_o.rename(dict(zip(raw_o.columns, new_headers)))

In [6]:
import unicodedata

def to_snake_case(name: str) -> str:
    """Converts a string to snake_case, handling accented characters."""
    name = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8')
    name = re.sub(r'[^\w\s]', '', name).strip().lower()
    return re.sub(r'\s+', '_', name)
alu = raw_o.filter(pl.col("DNI/Pasaporte alumno") != "DNI/Pasaporte alumno")
# 1. Rename columns to snake case and be more descriptive
rename_dict = {
    "CÓDIGO ESTUDIO": "codigo_estudio",
    "DESCRIPCIÓN ESTUDIO": "descripcion_estudio",
    " DESCRIPTOR AGRUPADO": "descriptor_agrupado",
    "NUEVO INGRESO?": "nuevo_ingreso",
    "Año académico": "ano_academico",
    "Año acceso": "ano_acceso",
    "DNI/Pasaporte alumno": "dni_pasaporte_alumno",
    "NIP alumno": "nip_alumno",
    "Nombre alumno": "nombre_alumno",
    "Apellido 1 alumno": "apellido1_alumno",
    "Apellido 2 alumno": "apellido2_alumno",
    "Apellidos (juntos)": "apellidos_alumno",
    "Centro desc.": "descripcion_centro",
    "Clase liquidación": "clase_liquidacion",
    "Comunidad habit": "comunidad_habitual",
    "Comunidad nacim": "comunidad_nacimiento",
    "Correo elec 2 alumno": "correo_electronico2_alumno",
    "Correo elec alumno": "correo_electronico_alumno",
    "Créditos matriculados": "creditos_matriculados",
    "Créditos reconocidos": "creditos_reconocidos",
    "Curso": "curso",
    "Curso más alto": "curso_mas_alto",
    "Edad": "edad",
    "Estado condicional": "estado_condicional",
    "Estado condicional original": "estado_condicional_original",
    "Fecha matrícula": "fecha_matricula",
    "Importe académico": "importe_academico",
    "Importe admin": "importe_administrativo",
    "Importe opc": "importe_opcional",
    "Localidad curso": "localidad_curso",
    "Localidad habit": "localidad_habitual",
    "Localidad nacim": "localidad_nacimiento",
    "País nacimiento": "pais_nacimiento",
    "País nacionalidad": "pais_nacionalidad",
    "Plan código": "plan_codigo",
    "Plan desc.": "descripcion_plan",
    "Programa intercambio": "programa_intercambio",
    "Provincia habit": "provincia_habitual",
    "Provincia nacim": "provincia_nacimiento",
    "Rama conocimiento": "rama_conocimiento",
    "Sexo alumno": "sexo_alumno",
    "Tipo acceso": "tipo_acceso",
    "Tipo beca": "tipo_beca",
    "Tipo estudio": "tipo_estudio",
    "Tipo matrícula": "tipo_matricula",
    "Turno": "turno",
    "Vía acceso": "via_acceso",
    "TIMESTAMP": "timestamp",
    "espanol_o_extranjero": "es_extranjero",
    "cartagena_campus?": "es_cartagena_campus",
    "online": "es_online",
}

# Apply the renaming
alu = raw_o.rename(rename_dict)
# For the columns not explicitly in the rename_dict, apply snake_case
for col in alu.columns:
    if col not in rename_dict.values():
        alu = alu.rename({col: to_snake_case(col)})



In [7]:
# 2. Change column types

alu = alu.with_columns([
    pl.col("nip_alumno").cast(pl.Int64),
    pl.col("ano_acceso").cast(pl.Int64),
    pl.col("ano_academico").cast(pl.Int64),
    # Handle creditos_matriculados as float
    pl.col("creditos_matriculados").str.replace(",", ".").cast(pl.Float64).alias("creditos_matriculados"),
    # Handle creditos_reconocidos as float
    pl.col("creditos_reconocidos").str.replace(",", ".").cast(pl.Float64).alias("creditos_reconocidos"),
    pl.col("curso").cast(pl.Int64),
    pl.col("curso_mas_alto").cast(pl.Int64),
    # Handle importe_administrativo as float
    pl.col("importe_administrativo").str.replace(",", ".").cast(pl.Float64).alias("importe_administrativo"),
    # Handle importe_opcional as float
    pl.col("importe_opcional").str.replace(",", ".").cast(pl.Float64).alias("importe_opcional"),
    pl.col("importe_academico").str.replace(".", "", literal=True).str.replace(",", ".").cast(pl.Float64).alias("importe_academico"),

    pl.col("plan_codigo").cast(pl.Int64),
    pl.when(pl.col("es_cartagena_campus") == "1").then(True)
      .when(pl.col("es_cartagena_campus") == "0").then(False)
      .otherwise(None).cast(pl.Boolean).alias("es_cartagena_campus"),
    pl.when(pl.col("es_online") == "TRUE").then(True)
      .when(pl.col("es_online") == "FALSE").then(False)
      .otherwise(None).cast(pl.Boolean).alias("es_online"),
    pl.when(pl.col("nuevo_ingreso") == "NUEVO INGRESO").then(True)
      .when(pl.col("nuevo_ingreso") == "RENOVACIÓN").then(False)
      .otherwise(None).cast(pl.Boolean).alias("nuevo_ingreso"),
    pl.when(pl.col("es_extranjero") == "extranjero").then(True)
      .when(pl.col("es_extranjero") == "nacional").then(False)
      .otherwise(None).cast(pl.Boolean).alias("es_extranjero"),
    pl.col("fecha_matricula").str.strptime(pl.Date, "%d/%m/%Y").alias("fecha_matricula"),
    pl.col("timestamp").str.strptime(pl.Datetime, "%d/%m/%Y %H:%M:%S:%3f").alias("timestamp"),
])

In [8]:
# 3. Delete specified columns
columns_to_drop = [
    "centro_cod",
    "centro_cou_cfgs",
    "centro_cou_cfgs_denom",
    "codigo_universidad_pau",
    "comarca_curso",
    "comarca_habit",
    "comarca_nacim",
    "comunidad_curso",
    "fecha_fichero",
    "orden_preferencia_preins",
    "matriculado",
    "matricula_anulada",
    "lote",
    "fichero",
    "fecha_matricula_firme",
    "pais_sistema_educacion_extranjero",
    "provincia_curso",
    "localidad_curso",
    "localidad_nacimiento",
    "sistema_educativo_extranjero",
]
alu = alu.drop(columns_to_drop)


In [9]:
alu = alu.with_columns(
    pl.concat_str([pl.col("nombre_alumno"), pl.lit(" "), pl.col("apellidos_alumno")]).alias("nombre_completo"))

##### Dataprep ESI + TitP

In [10]:
from datetime import datetime

In [11]:
# ESI

columnas_seleccionadas = [
    "Año Académico",
    "Denominación Estudio",
    "Denominación País 1",
    "Dni Pasaporte",
    "Email",
    "Email 2",
    "Fecha nacimiento alumno",
    "Nip Alumno",
    "Nombre Alumno",
    "Primer Apellido Alumno",
    "Segundo Apellido Alumno",
    "Número Créditos Matriculados",
    "Sexo Alumno",
    "Total Tasa Académica",
]

esi_trans = raw_esi.select(columnas_seleccionadas)

# Asumiendo que 'df_seleccionado' es el DataFrame resultante del paso anterior

# 1. Crear la columna "edad_alu"
esi_trans2 = esi_trans.with_columns(
    ((pl.lit(datetime.now().year) - pl.col("Fecha nacimiento alumno").dt.year()) -
     pl.when(
         (pl.lit(datetime.now().month) < pl.col("Fecha nacimiento alumno").dt.month()) |
         ((pl.lit(datetime.now().month) == pl.col("Fecha nacimiento alumno").dt.month()) &
          (pl.lit(datetime.now().day) < pl.col("Fecha nacimiento alumno").dt.day()))
     )
     .then(1)
     .otherwise(0)).alias("edad_alu")
)

esi_trans2 = esi_trans2.with_columns(
    pl.lit(True).alias("es_esi")
)

# 2. Cambiar el nombre de las columnas
nombres_nuevos = {
    "Año Académico": "ano_academico",
    "Denominación Estudio": "descripcion_estudio",
    "Denominación País 1": "pais_nacimiento",
    "Dni Pasaporte": "dni_pasaporte_alumno",
    "Email": "correo_electronico_alumno",
    "Email 2": "correo_electronico2_alumno",
    "Fecha nacimiento alumno": "fecha_nacimiento",
    "Nip Alumno": "nip_alumno",
    "Nombre Alumno": "nombre_alumno",
    "Primer Apellido Alumno": "apellido1_alumno",
    "Segundo Apellido Alumno": "apellido2_alumno",
    "Número Créditos Matriculados": "creditos_matriculados",
    "Sexo Alumno": "sexo_alumno",
    "Total Tasa Académica": "importe_academico",
}

esi_trans3 = esi_trans2.rename(nombres_nuevos)
esi = esi_trans3.with_columns(
    pl.col("ano_academico").cast(pl.Int64),
    pl.col("descripcion_estudio").cast(pl.String),
    pl.col("pais_nacimiento").cast(pl.String),
    pl.col("dni_pasaporte_alumno").cast(pl.String),
    pl.col("correo_electronico_alumno").cast(pl.String),
    pl.col("correo_electronico2_alumno").cast(pl.String),
    pl.col("fecha_nacimiento").cast(pl.Date),
    pl.col("nip_alumno").cast(pl.Int64),
    pl.col("nombre_alumno").cast(pl.String),
    pl.col("apellido1_alumno").cast(pl.String),
    pl.col("apellido2_alumno").cast(pl.String),
    pl.col("creditos_matriculados").cast(pl.Float64),
    pl.col("sexo_alumno").cast(pl.String),
    pl.col("importe_academico").str.replace(",", ".").cast(pl.Float64),
    pl.col("edad_alu").cast(pl.Int32),
    pl.col("es_esi").cast(pl.Boolean).alias("es_esi"),
)

In [12]:
# TITULOS PROPIOS


# Columnas que queremos seleccionar del DataFrame tp
columnas_seleccionadas_tp = [
    "Año Académico",
    "Denominación Estudio",
    "Denominación País 1",
    "Dni Pasaporte",
    "Email",
    "Email 2",
    "Fecha nacimiento alumno",
    "Nip Alumno",
    "Nombre Alumno",
    "Primer Apellido Alumno",
    "Segundo Apellido Alumno",
    "Número Créditos Matriculados",
    "Sexo Alumno",
    "Total Tasa Académica",
]

# 1. Seleccionar las columnas relevantes del DataFrame tp
try:
    tp_trans1 = raw_tp.select(columnas_seleccionadas_tp)
except pl.ColumnNotFoundError as e:
    print(f"Error: No se encontró la columna '{e}'. Asegúrate de que las columnas existan en el DataFrame tp.")
    exit()

# 2. Crear la columna "edad_alu"
tp_trans2 = tp_trans1.with_columns(
    ((pl.lit(datetime.now().year) - pl.col("Fecha nacimiento alumno").dt.year()) -
     pl.when(
         (pl.lit(datetime.now().month) < pl.col("Fecha nacimiento alumno").dt.month()) |
         ((pl.lit(datetime.now().month) == pl.col("Fecha nacimiento alumno").dt.month()) &
          (pl.lit(datetime.now().day) < pl.col("Fecha nacimiento alumno").dt.day()))
     )
     .then(1)
     .otherwise(0)).alias("edad_alu")
)
tp_trans2 = tp_trans2.with_columns(
    pl.lit(True).alias("es_titulopropio")
)

# 3. Cambiar el nombre de las columnas
nombres_nuevos_tp = {
    "Año Académico": "ano_academico",
    "Denominación Estudio": "descripcion_estudio",
    "Denominación País 1": "pais_nacimiento",
    "Dni Pasaporte": "dni_pasaporte_alumno",
    "Email": "correo_electronico_alumno",
    "Email 2": "correo_electronico2_alumno",
    "Fecha nacimiento alumno": "fecha_nacimiento",
    "Nip Alumno": "nip_alumno",
    "Nombre Alumno": "nombre_alumno",
    "Primer Apellido Alumno": "apellido1_alumno",
    "Segundo Apellido Alumno": "apellido2_alumno",
    "Número Créditos Matriculados": "creditos_matriculados",
    "Sexo Alumno": "sexo_alumno",
    "Total Tasa Académica": "importe_academico",
}

tp_trans3 = tp_trans2.rename(nombres_nuevos_tp)

# 4. Castear las columnas al tipo de dato correcto
tit_propios = tp_trans3.with_columns(
    pl.col("ano_academico").cast(pl.Int64),
    pl.col("descripcion_estudio").cast(pl.String),
    pl.col("pais_nacimiento").cast(pl.String),
    pl.col("dni_pasaporte_alumno").cast(pl.String),
    pl.col("correo_electronico_alumno").cast(pl.String),
    pl.col("correo_electronico2_alumno").cast(pl.String),
    pl.col("fecha_nacimiento").cast(pl.Date),
    pl.col("nip_alumno").cast(pl.Int64),
    pl.col("nombre_alumno").cast(pl.String),
    pl.col("apellido1_alumno").cast(pl.String),
    pl.col("apellido2_alumno").cast(pl.String),
    pl.col("creditos_matriculados").cast(pl.Float64),
    pl.col("sexo_alumno").cast(pl.String),
    pl.col("importe_academico").str.replace(",", ".").cast(pl.Float64),
    pl.col("edad_alu").cast(pl.Int32),
    pl.col("es_titulopropio").cast(pl.Boolean).alias("es_titulopropio"),
)

In [13]:
titp = pl.concat([esi, tit_propios], how="diagonal")
titp = titp.with_columns(
    pl.concat_str([pl.col("nombre_alumno"), pl.lit(" "), pl.col("apellido1_alumno"), pl.lit(" "), pl.col("apellido2_alumno")]).alias("nombre_completo"))

##### **aluf merging** (titp+oficial)

In [14]:
aluf = pl.concat([titp, alu], how="diagonal")

diccionario país-continente 👇

In [15]:
# @title
mapa_continentes = {
    'Polonia': 'Europa',
    'Chipre': 'Europa',
    'San Cristobal y Nieves': 'América',
    'Tailandia': 'Asia',
    'Etiopía': 'África',
    'México': 'América',
    'Reunión': 'África',
    'Grecia': 'Europa',
    'Italia': 'Europa',
    'Luxemburgo': 'Europa',
    'Trinidad y Tobago': 'América',
    'Rusia': 'Europa',  # Consideramos la parte europea de Rusia
    'Mauricio(islas)': 'África',
    'Alemania': 'Europa',
    'Filipinas': 'Asia',
    'Sri Lanka': 'Asia',
    'Portugal': 'Europa',
    'Haití': 'América',
    'Líbano': 'Asia',
    'Zimbabwe (Rodesia)': 'África',
    'Australia': 'Oceanía',
    'Uruguay': 'América',
    'Swazilandia': 'África',
    'Bulgaria': 'Europa',
    'Burundi': 'África',
    'Sudáfrica': 'África',
    'Aruba': 'América',
    'Bielorrusia': 'Europa',
    'Holanda': 'Europa',  # También conocido como Países Bajos
    'Jamaica': 'América',
    'El Salvador': 'América',
    'Colombia': 'América',
    'República Islámica de Irán': 'Asia',
    'Bolivia': 'América',
    'Brasil': 'América',
    'Polinesia Francesa': 'Oceanía',
    'Andorra': 'Europa',
    'Cabo Verde': 'África',
    'Rumanía': 'Europa',
    'Eslovenia': 'Europa',
    'Azerbayán': 'Asia',  # Consideramos Azerbaiyán como parte de Asia
    'Guatemala': 'América',
    'Djibuti': 'África',
    'Turquía': 'Europa', # Aunque tiene parte en Asia, cultural y geográficamente se suele incluir en Europa en muchos contextos
    'Perú': 'América',
    'España': 'Europa',
    'Burkina-Faso (Alto Volta)': 'África',
    'Congo': 'África',
    'Lituania': 'Europa',
    'Bangladesh': 'Asia',
    'Ucrania': 'Europa',
    'Libia': 'África',
    'Suiza': 'Europa',
    'Rep. Democrática del Congo': 'África',
    'Finlandia': 'Europa',
    'Hong Kong': 'Asia',
    'Eslovaquia': 'Europa',
    'Islandia': 'Europa',
    'Túnez': 'África',
    'Angola': 'África',
    'Cuba': 'América',
    'Serbia': 'Europa',
    'Antillas Holandesas': 'América',
    'Gambia': 'África',
    'Camboya': 'Asia',
    'Estados Unidos de América': 'América',
    'Japón': 'Asia',
    'Malasia': 'Asia',
    'Honduras': 'América',
    'Paraguay': 'América',
    'Bosnia Herzegovina': 'Europa',
    'Senegal': 'África',
    'Marruecos': 'África',
    'Irán': 'Asia',
    'República Dominicana': 'América',
    'Ecuador': 'América',
    'Macao': 'Asia',
    'Bélgica': 'Europa',
    'Guinea': 'África',
    'Vietnam': 'Asia',
    'Venezuela': 'América',
    'Noruega': 'Europa',
    'República Checa': 'Europa',
    'Bahrein': 'Asia',
    'Martinica': 'América',
    'Mongolia': 'Asia',
    'Mozambique': 'África',
    'Kirguistán': 'Asia',
    'Costa Rica': 'América',
    'Corea del Sur': 'Asia',
    'Irlanda': 'Europa',
    'Puerto Rico': 'América',
    'Ghana': 'África',
    'Níger': 'África',
    'Namibia': 'África',
    'Nigeria': 'África',
    'Croacia': 'Europa',
    'Uganda': 'África',
    'Sáhara Occidental': 'África',
    'Camerún': 'África',
    'Myanmar': 'Asia',
    'Argentina': 'América',
    'Dominica': 'América',
    'ALAN': None,  # No se reconoce este país, puedes investigarlo o asignarle un continente si lo conoces
    'Egipto': 'África',
    'Estonia': 'Europa',
    'Pakistán': 'Asia',
    'Estado de Palestina': 'Asia',
    'Sudán del Norte': 'África',
    'Qatar': 'Asia',
    'Ruanda': 'África',
    'Argelia': 'África',
    'Kenia': 'África',
    'Países Bajos': 'Europa', # Otra forma de referirse a Holanda
    'Armenia': 'Asia', # Consideramos Armenia como parte de Asia
    'Siria': 'Asia',
    'Costa de Marfil': 'África',
    'Nepal': 'Asia',
    'Tanzania': 'África',
    'India': 'Asia',
    'República de San Marino': 'Europa',
    'Suecia': 'Europa',
    'Nicaragua': 'América',
    'Guinea Ecuatorial': 'África',
    'Guinea Bissau': 'África',
    'Uzbekistán': 'Asia',
    'Panamá': 'América',
    'Chile': 'América',
    'Guyana': 'América',
    'Nueva Caledonia': 'Oceanía',
    'Dinamarca': 'Europa',
    'Nueva Zelanda': 'Oceanía',
    'Israel': 'Asia',
    'Hungría': 'Europa',
    'Moldavia': 'Europa',
    'Canadá': 'América',
    'Arabia Saudita': 'Asia',
    'Taiwán': 'Asia',
    'Austria': 'Europa',
    'Georgia': 'Asia', # Consideramos Georgia como parte de Asia
    'Albania': 'Europa',
    'Irak': 'Asia',
    'Letonia': 'Europa',
    'Reino Unido': 'Europa',
    'Jordania': 'Asia',
    'Indonesia': 'Asia',
    'Francia': 'Europa',
    'China': 'Asia',
    'Mónaco': 'Europa',
    'Montenegro': 'Europa',
    'Kuwait': 'Asia',
    'Afganistán': 'Asia',
    'Corea del Norte': 'Asia',
    'Emiratos Árabes Unidos': 'Asia',
    'Bahamas': 'América',
    'Taiwan': 'Asia', # Otra forma de referirse a Taiwán
    'Kazjastan': 'Asia' # Consideramos Kazajistán como parte de Asia
}

# Suponiendo que ya tienes un DataFrame llamado 'alu' con una columna 'pais_nacimiento'
aluf = aluf.with_columns(
    pl.col("pais_nacimiento")
      .map_elements(lambda x: mapa_continentes.get(x),
                   return_dtype=pl.Utf8,
                   skip_nulls=True)
      .alias("continente_nacimiento")
)

In [16]:
aluf=aluf.with_columns(
    pl.col("timestamp").is_not_null().alias("es_oficial")
)
aluf=aluf.drop("es_extranjero")
aluf = aluf.with_columns(
    pl.when(pl.col("pais_nacimiento") == "España")
    .then(False)
    .otherwise(True)
    .alias("es_extranjero")
)

In [17]:
aluf = aluf.with_columns(
    [
        pl.col("es_esi").fill_null(False),
        pl.col("es_oficial").fill_null(False),
        pl.col("es_titulopropio").fill_null(False),
    ]
)

# Creación BBDD (SQLite)

In [ ]:
import sqlite3
import os

ruta_drive = '/content/drive/MyDrive/BBDDAlumnos/'
nombre_db = 'aluf_relacional.db'
ruta_completa_db = os.path.join(ruta_drive, nombre_db)

conn = sqlite3.connect(ruta_completa_db)
cursor = conn.cursor()

In [ ]:
conn = sqlite3.connect(ruta_completa_db)
cursor = conn.cursor()

# Crear la tabla 'alumnos'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS alumnos (
        id_alumno INTEGER PRIMARY KEY AUTOINCREMENT,
        dni TEXT,
        nom_completo TEXT,
        edad INTEGER,
        sexo TEXT,
        cred_matriculados REAL,
        email_alu TEXT,
        email_personal TEXT,
        pais_nac TEXT,
        continente_nac TEXT,
        importe REAL,
        curso_matricula INTEGER,
        id_estudio TEXT,
        FOREIGN KEY (id_estudio) REFERENCES estudios(id_estudio),
        UNIQUE (dni, curso_matricula, id_estudio)
    )
''')

# Crear la tabla 'estudios'
cursor.execute('''
    CREATE TABLE IF NOT EXISTS estudios (
        id_estudio TEXT PRIMARY KEY,
        id_facultad INTEGER,
        es_esi BOOLEAN,
        es_titpropio BOOLEAN,
        es_online BOOLEAN,
        tipo_estudio TEXT,
        campus TEXT,
        FOREIGN KEY (id_facultad) REFERENCES facultades(id_facultad)
    )
''')

conn.commit()

In [ ]:
def extraer_datos_e_insertar(df: pl.DataFrame, cursor: sqlite3.Cursor):
    alumnos_data = []
    estudios_data = []

    for row in df.rows(named=True):
        # Datos para la tabla 'alumnos'
        alumnos_data.append((
            row['dni_pasaporte_alumno'],
            row['nombre_completo'],
            row['edad_alu'],
            row['sexo_alumno'],
            row['creditos_matriculados'],
            row['correo_electronico_alumno'],
            row['correo_electronico2_alumno'],
            row['pais_nacimiento'],
            row['continente_nacimiento'],
            row['importe_academico'],
            row['ano_academico'],
            row['codigo_estudio']
        ))

        # Lógica para determinar 'es_online'
        descripcion_estudio = row.get('descripcion_estudio')
        es_online = False
        if descripcion_estudio is not None and isinstance(descripcion_estudio, str):
            es_online = "a distancia" in descripcion_estudio.lower()

        # Lógica para determinar 'tipo_estudio'
        tipo_estudio = row.get('tipo_estudio')
        if not tipo_estudio:
            descripcion_estudio = row.get('descripcion_estudio')
            if descripcion_estudio is not None and isinstance(descripcion_estudio, str):
                if "máster permanente" in descripcion_estudio.lower():
                    tipo_estudio = "master_p"
                elif "diploma" in descripcion_estudio.lower():
                    tipo_estudio = "microcredencial"
            elif row.get('es_esi'):
                tipo_estudio = "idioma"
            else:
                tipo_estudio = None

        # Lógica para determinar 'campus'
        descriptor_agrupado = row.get('descriptor_agrupado')
        campus = None
        if descriptor_agrupado is not None and isinstance(descriptor_agrupado, str):
            if "cartagena" in descriptor_agrupado.lower():
                campus = "Cartagena"
            elif "murcia" in descriptor_agrupado.lower():
                campus = "Murcia"
            elif "madrid" in descriptor_agrupado.lower():
                campus = "Madrid"

        # Datos para la tabla 'estudios'
        estudios_data.append((
            row['codigo_estudio'],
            None,  # id_facultad - lo relacionaremos después
            row['es_esi'],
            row['es_titpropio'],
            es_online,
            tipo_estudio,
            campus
        ))

    # Insertar datos en la tabla 'alumnos'
    cursor.executemany("INSERT OR IGNORE INTO alumnos (dni, nom_completo, edad, sexo, cred_matriculados, email, pais_nac, continente_nac, importe, curso_matricula, id_estudio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", alumnos_data)

    # Insertar datos en la tabla 'estudios'
    cursor.executemany("INSERT OR IGNORE INTO estudios VALUES (?, ?, ?, ?, ?, ?, ?)", estudios_data)

# Asumiendo que 'df_aluf' ya está cargado
# extraer_datos_e_insertar(df_aluf, cursor)
# conn.commit() # Movemos el commit al final para agrupar las operaciones

In [ ]:
ruta_facultades_tsv = '/content/drive/MyDrive/BBDDAlumnos/facultades.tsv'

try:
    df_facultades = pl.read_csv(ruta_facultades_tsv, separator='\t')

    # Convertir la columna 'titulaciones' a una lista dividiendo por '|'
    df_facultades = df_facultades.with_columns(
        pl.col("titulaciones").str.split("\\|")
    )

    # Crear la tabla 'facultades' - CORRECCIÓN EN LOS NOMBRES DE LAS COLUMNAS
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS facultades (
            id_facultad TEXT PRIMARY KEY,
            nom_facultad TEXT UNIQUE,
            decano TEXT,
            titulaciones TEXT
        )
    ''')

    # Insertar datos en la tabla 'facultades'
    for row in df_facultades.rows(named=True):
        try:
            cursor.execute('''
                INSERT OR IGNORE INTO facultades (id_facultad, nom_facultad, decano, titulaciones)
                VALUES (?, ?, ?, ?)
            ''', (row['id_facultad'], row['nom_facultad'], row['decano'], '|'.join(row['titulaciones'])))
        except sqlite3.Error as e:
            print(f"Error al insertar la facultad {row['id_facultad']}: {e}")

    conn.commit()

except Exception as e:
    print(f"Error al procesar el archivo de facultades: {e}")

In [ ]:
    valores_unicos_centro = aluf.select(pl.col("descripcion_estudio")).unique()
    with pl.Config(fmt_str_lengths=120,  # mostrar hasta 1000 caracteres :contentReference[oaicite:5]{index=5}
               tbl_rows=-1):   # sin límite de cantidad de filas
               print(valores_unicos_centro)


shape: (446, 1)
┌──────────────────────────────────────────────────────────────────────────────────────────────────┐
│ descripcion_estudio                                                                              │
│ ---                                                                                              │
│ str                                                                                              │
╞══════════════════════════════════════════════════════════════════════════════════════════════════╡
│ CURSO INTENSIVO VERANO B1 UCAM                                                                   │
│ Curso de formación para el acceso a la Universidad de mayores de 25 y 45 años                    │
│ Curso superior universitario en Técnicas de Cirugía Ortopédica y Traumatología                   │
│ Máster Universitario en Enseñanza Bilingüe: Inglés                                               │
│ DECA - Módulo de Religión Católica y su Pedagogía para la obtención de la

In [ ]:
aluf

ano_academico,descripcion_estudio,pais_nacimiento,dni_pasaporte_alumno,correo_electronico_alumno,correo_electronico2_alumno,fecha_nacimiento,nip_alumno,nombre_alumno,apellido1_alumno,apellido2_alumno,creditos_matriculados,sexo_alumno,importe_academico,edad_alu,es_esi,es_titulopropio,nombre_completo,codigo_estudio,descriptor_agrupado,nuevo_ingreso,ano_acceso,apellidos_alumno,descripcion_centro,clase_liquidacion,comunidad_habitual,comunidad_nacimiento,creditos_reconocidos,curso,curso_mas_alto,edad,estado_condicional,estado_condicional_original,fecha_matricula,importe_administrativo,importe_opcional,localidad_habitual,pais_nacionalidad,plan_codigo,descripcion_plan,programa_intercambio,provincia_habitual,provincia_nacimiento,rama_conocimiento,tipo_acceso,tipo_beca,tipo_estudio,tipo_matricula,turno,via_acceso,timestamp,es_cartagena_campus,es_online,continente_nacimiento,es_oficial,es_extranjero
i64,str,str,str,str,str,date,i64,str,str,str,f64,str,f64,i32,bool,bool,str,str,str,bool,i64,str,str,str,str,str,f64,i64,i64,str,str,str,date,f64,f64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,datetime[ms],bool,bool,str,bool,bool
2020,"""CURSO ONLINE DE ESPAÑOL B2.4""","""Irlanda""","""PV3279885""","""mmoshaughnessy@alu.ucam.edu""","""meganoshaughnessy97@gmail.com""",1997-01-21,92251,"""Megan Mary""","""O Shaughnessy""",""" """,5.0,"""F""",125.0,28,true,false,"""Megan Mary O Shaughnessy """,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Europa""",false,true
2020,"""INGLÉS CURSO FORMACIÓN B2 SÁBA…","""España""","""48421867J""","""amgarrido@ucam.edu""","""amgarrido@ucam.edu""",1979-11-11,43312,"""Antonio""","""Martínez""","""Garrido""",3.0,"""M""",450.0,45,true,false,"""Antonio Martínez Garrido""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Europa""",false,false
2020,"""CURSO CAE 2ªº CUATRIM.""","""España""","""23953498X""","""cefernandez@ucam.edu""","""eugevaleros29@gmail.com""",1993-12-29,28328,"""Clara Eugenia""","""Fernández""","""Valeros""",3.0,"""F""",450.0,31,true,false,"""Clara Eugenia Fernández Valero…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Europa""",false,false
2020,"""CURSO FUNDAE C1 SÁBADOS""","""Italia""","""X8426948R""","""mbruno@ucam.edu""","""m.bruno84@gmail.com""",1984-12-12,70941,"""Marco""","""Bruno""","""Bruno""",3.0,"""M""",0.0,40,true,false,"""Marco Bruno Bruno""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Europa""",false,true
2020,"""INTENSIVO B1 DE MAYO A JUNIO 2…","""España""","""49194081W""","""fblaya2@alu.ucam.edu""","""mmiralles@ucam.edu""",1998-08-16,69431,"""Fernando""","""Blaya""","""Miralles""",3.0,"""M""",375.0,26,true,false,"""Fernando Blaya Miralles""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Europa""",false,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024,"""Graduado o Graduada en Odontol…","""República Islámica de Irán""","""Z97575352""","""fshaarbafi@alu.ucam.edu""","""fatemehshaarbafi@gmail.com""",null,121546,"""Fatemeh""","""Shaarbafi""",null,84.0,"""F""",20510.28,null,false,false,"""Fatemeh Shaarbafi ""","""89""","""GRADO EN ODONTOLOGÍA MURCIA GE…",false,2021,"""Shaarbafi ""","""FACULTAD DE CIENCIAS DE LA SAL…","""Sin bonificación""","""Región de Murcia""","""Sin C.A. asociada""",0.0,4,4,"""De 31 a 35""",null,null,2024-07-25,85.0,0.0,"""La Ñora""","""Repú

In [ ]:
def extraer_datos(df: pl.DataFrame):
    alumnos_data = []
    estudios_data = []

    for row in df.rows(named=True):
        # Datos para la tabla 'alumnos'
        alumnos_data.append((
            row['dni_pasaporte_alumno'],
            row['nombre_completo'],
            row['edad_alu'],
            row['sexo_alumno'],
            row['creditos_matriculados'],
            row['correo_electronico_alumno'],
            row['pais_nacimiento'],
            row['continente_nacimiento'],
            row['importe_academico'],
            row['ano_academico'],
            row['codigo_estudio']
        ))

        # Lógica para determinar 'es_online'
        descripcion_estudio = row.get('descripcion_estudio')
        es_online = False
        if descripcion_estudio is not None and isinstance(descripcion_estudio, str):
            es_online = "a distancia" in descripcion_estudio.lower()

        # Lógica para determinar 'tipo_estudio'
        tipo_estudio = row.get('tipo_estudio')
        if not tipo_estudio:
            descripcion_plan = row.get('descripcion_plan')
            if descripcion_plan is not None and isinstance(descripcion_plan, str):
                if "máster permanente" in descripcion_plan.lower():
                    tipo_estudio = "master_p"
                elif "diploma" in descripcion_plan.lower():
                    tipo_estudio = "microcredencial"
            elif row.get('es_esi'):
                tipo_estudio = "idioma"
            else:
                tipo_estudio = None # O podrías asignar un valor por defecto

        # Lógica para determinar 'campus'
        descripcion_plan = row.get('descripcion_plan')
        campus = None
        if descripcion_plan is not None and isinstance(descripcion_plan, str):
            if "cartagena" in descripcion_plan.lower():
                campus = "Cartagena"
            elif "murcia" in descripcion_plan.lower():
                campus = "Murcia"
            elif "madrid" in descripcion_plan.lower():
                campus = "Madrid"

        # Datos para la tabla 'estudios'
        estudios_data.append((
            row['codigo_estudio'],
            None,  # id_facultad - lo relacionaremos después
            row['es_esi'],
            row['es_titulopropio'],
            es_online,
            tipo_estudio,
            campus
        ))

    return alumnos_data, estudios_data

In [ ]:
alumnos_data, estudios_data = extraer_datos(aluf)

# Insertar datos en la tabla 'alumnos'
cursor.executemany("INSERT OR IGNORE INTO alumnos (dni, nom_completo, edad, sexo, cred_matriculados, email, pais_nac, continente_nac, importe, curso_matricula, id_estudio) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", alumnos_data)

# Insertar datos en la tabla 'estudios'
cursor.executemany("INSERT OR IGNORE INTO estudios VALUES (?, ?, ?, ?, ?, ?, ?)", estudios_data)

In [ ]:
ruta_facultades_tsv = '/content/drive/MyDrive/facultades.tsv'

In [ ]:
# Ejemplo de cómo relacionar las tablas 'estudios' y 'facultades'
facultad_mapping = {
    "Escuela de Ingeniería de Telecomunicación": 1,
    "Facultad de Informática": 2,
    "Escuela Técnica Superior de Ingeniería Industrial": 3,
    # ... añade más facultades y sus IDs correspondientes
}

for row in aluf.rows(named=True):
    descripcion_centro = row['descripcion_centro']
    codigo_estudio = row['codigo_estudio']
    for nombre_facultad, id_facultad in facultad_mapping.items():
        if nombre_facultad.lower() in descripcion_centro.lower():
            cursor.execute("UPDATE estudios SET id_facultad = ? WHERE id_estudio = ?", (id_facultad, codigo_estudio))
            break

# Importación de Datos

In [ ]:
aluf=aluf.lazy()

# Options and Settings

`**aluf**` es un lazyframe para mejorar el rendimiento de las queries y que sea más fácil generar nuevas queries con chatbots

In [ ]:
query=(
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("es_oficial") == True)
    .group_by("descripcion_plan")
    .agg(pl.count().alias("n_alumnos"))
    .sort("n_alumnos", descending=True)
)
query

# PONER ESTO AL FINAL DE UNA QUERY (se hace print final de la query)PARA QUE SE PUEDAN VER TODAS LAS FILAS.
with pl.Config(fmt_str_lengths=120,  # mostrar hasta 1000 caracteres :contentReference[oaicite:5]{index=5}
               tbl_rows=-1):   # sin límite de cantidad de filas
  print(query_master)

In [ ]:
# Alternativamente se puede hacer:
pandas_df = [DATAFRAME]df.to_pandas()
from google.colab import sheets
sh = sheets.InteractiveSheet(df=pandas_df)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# DEPRECATED
# pl.Config.display_max_width = 200
# pl.Config.display_max_columns = 100
# pl.Config.display_max_rows = 200

# Queries

In [ ]:
aluf_pd=aluf.collect().to_pandas()
aluf_pd

##### **QUERIES RELEVANTES**

In [ ]:
### Para buscar alumnos (usando nombres)

# - sensible a tildes/acentos
# - devuelve: nombre, estudios, año, emails
# INTRODUCIR NOMBRE O APELLIDO EN str.contains("___",literal=False)

query25 = aluf.select([
    pl.col("nombre_completo").alias("nombre"),
    pl.col("descripcion_estudio").alias("estudio"),
    pl.col("ano_academico").alias("año_academico"),
    pl.col("correo_electronico_alumno").alias("email_alu"),
    pl.col("correo_electronico2_alumno").alias("email_personal"),
]).filter(pl.col("nombre").str.to_lowercase().str.contains("pérez|perez", literal=False))
query25d=query25.collect()
print(query25d)

In [ ]:
aluf.schema

Schema([('ano_academico', Int64),
        ('descripcion_estudio', String),
        ('pais_nacimiento', String),
        ('dni_pasaporte_alumno', String),
        ('correo_electronico_alumno', String),
        ('correo_electronico2_alumno', String),
        ('fecha_nacimiento', Date),
        ('nip_alumno', Int64),
        ('nombre_alumno', String),
        ('apellido1_alumno', String),
        ('apellido2_alumno', String),
        ('creditos_matriculados', Float64),
        ('sexo_alumno', String),
        ('importe_academico', Float64),
        ('edad_alu', Int32),
        ('es_esi', Boolean),
        ('es_titulopropio', Boolean),
        ('nombre_completo', String),
        ('codigo_estudio', String),
        ('descriptor_agrupado', String),
        ('nuevo_ingreso', Boolean),
        ('ano_acceso', Int64),
        ('apellidos_alumno', String),
        ('descripcion_centro', String),
        ('clase_liquidacion', String),
        ('comunidad_habitual', String),
        ('comunidad_na

#### Nuevas queries

In [ ]:
# Buscar estudio por cantidad de ediciones que ha tenido (LIMITADO DESDE 2019 PARA ADELANTE)
query_master = aluf.select([
    pl.col("ano_academico").alias("año_academico"),
    pl.col("descripcion_estudio").alias("descripcion_estudio"),
]).filter(
    pl.col("descripcion_estudio").str.to_lowercase().str.contains("enfermería de urgencias", literal=False)
).unique().sort("año_academico")

query_master=query_master.collect()
print(query_master)

shape: (6, 2)
┌───────────────┬─────────────────────────────────┐
│ año_academico ┆ descripcion_estudio             │
│ ---           ┆ ---                             │
│ i64           ┆ str                             │
╞═══════════════╪═════════════════════════════════╡
│ 2019          ┆ Máster Universitario en Enferm… │
│ 2020          ┆ Máster Universitario en Enferm… │
│ 2021          ┆ Máster Universitario en Enferm… │
│ 2022          ┆ Máster Universitario en Enferm… │
│ 2023          ┆ Máster Universitario en Enferm… │
│ 2024          ┆ Máster Universitario en Enferm… │
└───────────────┴─────────────────────────────────┘


In [ ]:
# Pa.ra un DataFrame (no LazyFrame)
# 1 Agrupamos por alumno y obtenemos tipos de estudio únicos
alumnos_con_tipos = (
    aluf
    .group_by("dni_pasaporte_alumno")
    .agg([
        # Obtenemos lista de tipos de estudio únicos en minúsculas
        pl.col("tipo_estudio").str.to_lowercase().unique().alias("tipos_estudio"),
        # Contamos cuántos tipos diferentes hay
        pl.col("tipo_estudio").n_unique().alias("num_tipos")
    ])
)

# 2. Identificamos las combinaciones específicas
alumnos_clasificados = (
    alumnos_con_tipos
    .with_columns(
        pl.when(
            pl.col("tipos_estudio").list.contains("grado") &
            pl.col("tipos_estudio").list.contains("máster")
        ).then(pl.lit("Grado y Máster"))
        .when(
            pl.col("tipos_estudio").list.contains("grado") &
            pl.col("tipos_estudio").list.contains("doctorado")
        ).then(pl.lit("Grado y Doctorado"))
        .otherwise(pl.lit("Otros"))
        .alias("combinacion_estudios")
    )
)

# 3. Filtramos solo los casos que nos interesan
resultado_final = (
    alumnos_clasificados
    .filter(
        (pl.col("combinacion_estudios") == "Grado y Máster") |
        (pl.col("combinacion_estudios") == "Grado y Doctorado")
    )
    .sort(["combinacion_estudios", "dni_pasaporte_alumno"])
)

# Recuento de combinaciones de estudios
recuento_combinaciones = (
    resultado_final
    .group_by("combinacion_estudios")
    .agg(
        pl.count().alias("total_alumnos")
    )
    .sort("total_alumnos", descending=True)
)
recuento_combinaciones2=recuento_combinaciones.collect()
print(recuento_combinaciones2)

<ipython-input-42-dbcfab48b5bf>:46: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("total_alumnos")


shape: (2, 2)
┌──────────────────────┬───────────────┐
│ combinacion_estudios ┆ total_alumnos │
│ ---                  ┆ ---           │
│ str                  ┆ u32           │
╞══════════════════════╪═══════════════╡
│ Grado y Máster       ┆ 1239          │
│ Grado y Doctorado    ┆ 44            │
└──────────────────────┴───────────────┘


In [ ]:
# Recuento de combinaciones de estudios
recuento_combinaciones = (
    resultado_final
    .group_by("combinacion_estudios")
    .agg(
        pl.count().alias("total_alumnos")
    )
    .sort("total_alumnos", descending=True)
)

recuento_combinaciones=recuento_combinaciones.collect()
print(recuento_combinaciones)

<ipython-input-46-9530d50c351d>:6: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("total_alumnos")


shape: (2, 2)
┌──────────────────────┬───────────────┐
│ combinacion_estudios ┆ total_alumnos │
│ ---                  ┆ ---           │
│ str                  ┆ u32           │
╞══════════════════════╪═══════════════╡
│ Grado y Máster       ┆ 1239          │
│ Grado y Doctorado    ┆ 44            │
└──────────────────────┴───────────────┘


In [ ]:
# Porcentaje de alumnos internacionales por continente
query26 = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("es_extranjero") == True)
    .group_by("continente_nacimiento")
    .agg(pl.len().alias("n_alumnos"))
    .with_columns(
        (pl.col("n_alumnos") / pl.sum("n_alumnos").over(pl.lit(1)) * 100).round(2).alias("porcentaje"))
    .select(["continente_nacimiento", "porcentaje"])
)
query26=query26.collect()
print(query26)

shape: (6, 2)
┌───────────────────────┬────────────┐
│ continente_nacimiento ┆ porcentaje │
│ ---                   ┆ ---        │
│ str                   ┆ f64        │
╞═══════════════════════╪════════════╡
│ África                ┆ 6.08       │
│ Europa                ┆ 51.07      │
│ Asia                  ┆ 11.42      │
│ Oceanía               ┆ 0.24       │
│ América               ┆ 30.69      │
│ null                  ┆ 0.5        │
└───────────────────────┴────────────┘


In [ ]:
# pivot table para ver los alumnos de ESI y Titulos propios divididos entre alumnos nacionales e internacionales
query28 = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("es_oficial") == True)
    .pivot(
        index='es_esi',  # La columna que será el índice de las filas
        on='es_extranjero', # <-- Usa 'on' en lugar de 'columns'
        values='es_esi',      # <-- La columna de valores a agregar (puedes usar 'es_esi' o cualquier otra si solo quieres contar)
        aggregate_function='len' # <-- Usa 'len' en lugar de 'count' para contar filas,
    )
)
query28

In [ ]:
query_master = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("descripcion_plan").str.to_lowercase().str.contains("máster", literal=False))

    .group_by(pl.col("descripcion_plan").alias("titulación"))
    .agg(pl.len().alias("n_alumnos"))
    .select([
        pl.col("titulación"),
        pl.col("n_alumnos"),
    ])
)
with pl.Config(fmt_str_lengths=120,  # mostrar hasta 1000 caracteres :contentReference[oaicite:5]{index=5}
               tbl_rows=-1):   # sin límite de ancho de tabla :contentReference[oaicite:6]{index=6}
  print(query_master)

In [ ]:
from functools import reduce
import polars as pl

# Lista de palabras clave
keywords = [
    "Alto Rendimiento Deportivo",
    "Dirección y Gestión de Entidades Deportivas",
    "Enfermería de Urgencias",
    "Gestión Administrativa",
    "Medicina de Urgencias",
    "Fisioterapia en el Deporte",
    "Nutrición en la Actividad Física"
]

# Crear expresión compuesta usando reduce y operador OR
filtro = reduce(
    lambda acc, k: acc | pl.col("descripcion_plan").str.contains(k, literal=True),
    keywords[1:],
    pl.col("descripcion_plan").str.contains(keywords[0], literal=True)
)

# Aplicar el filtro
alumnos_filtrados = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(filtro)
    .group_by("descripcion_plan")
    .agg(pl.count().alias("n_alumnos"))
    .sort("n_alumnos", descending=True)
)

# Mostrar sin truncar
with pl.Config(fmt_str_lengths=1000, tbl_width_chars=None):
    print(alumnos_filtrados)


shape: (9, 2)
┌──────────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ descripcion_plan                                                                     ┆ n_alumnos │
│ ---                                                                                  ┆ ---       │
│ str                                                                                  ┆ u32       │
╞══════════════════════════════════════════════════════════════════════════════════════╪═══════════╡
│ Máster Universitario en Gestión Administrativa - Murcia - A Distancia - Castellano   ┆ 43        │
│ Máster Universitario en Alto Rendimiento Deportivo: Fuerza y Acondicionamiento       ┆ 41        │
│ Físico / Master in High Performance Sport: Strength and Conditioning - Murcia -      ┆           │
│ Semipresencial - Castellano                                                          ┆           │
│ Máster Universitario en Enfermería de Urgencias, Emergencias y Cuidados Esp

<ipython-input-57-68a42f7ceeca>:28: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("n_alumnos"))


In [ ]:
query_master = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("descriptor_agrupado").str.to_lowercase().str.contains("a distancia", literal=False))
    .filter(pl.col("tipo_estudio").str.to_lowercase().str.contains("máster", literal=False)) # Nuevo filtro para máster
    .group_by(pl.col("descriptor_agrupado").alias("titulación"))
    .agg(pl.count().alias("n_alumnos"))
    .select([
        pl.col("titulación"),
        pl.col("n_alumnos"),
        pl.lit(2024).alias("año_academico")
    ])
    .sort("titulación")
)

query_total = (
    query_master
    .select([
        pl.lit("TOTALES").alias("titulación"),
        pl.sum("n_alumnos").alias("n_alumnos"),
        pl.lit(2024).alias("año_academico")
    ])
)

query_final = pl.concat([query_master, query_total])
query_final

<ipython-input-43-be775f2b22fc>:7: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("n_alumnos"))


titulación,n_alumnos,año_academico
str,u32,i32
"""MURCIA GENERAL A DISTANCIA""",1104,2024
"""TOTALES""",1104,2024


In [ ]:
print(aluf)

shape: (105_908, 56)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ ano_acade ┆ descripci ┆ pais_naci ┆ dni_pasap ┆ … ┆ es_online ┆ continent ┆ es_oficia ┆ es_extra │
│ mico      ┆ on_estudi ┆ miento    ┆ orte_alum ┆   ┆ ---       ┆ e_nacimie ┆ l         ┆ njero    │
│ ---       ┆ o         ┆ ---       ┆ no        ┆   ┆ bool      ┆ nto       ┆ ---       ┆ ---      │
│ i64       ┆ ---       ┆ str       ┆ ---       ┆   ┆           ┆ ---       ┆ bool      ┆ bool     │
│           ┆ str       ┆           ┆ str       ┆   ┆           ┆ str       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2020      ┆ CURSO     ┆ Irlanda   ┆ PV3279885 ┆ … ┆ null      ┆ Europa    ┆ false     ┆ true     │
│           ┆ ONLINE DE ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ ESPAÑOL   ┆           ┆           ┆   ┆           ┆     

In [ ]:
# pivot table para ver los alumnos de ESI y Titulos propios divididos entre alumnos nacionales e internacionales
query28 = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("es_oficial") == False)
    .pivot(
        index='es_esi',  # La columna que será el índice de las filas
        on='es_extranjero', # <-- Usa 'on' en lugar de 'columns'
        values='es_esi',      # <-- La columna de valores a agregar (puedes usar 'es_esi' o cualquier otra si solo quieres contar)
        aggregate_function='len' # <-- Usa 'len' en lugar de 'count' para contar filas,
    )
)
query28

es_esi,false,true
bool,u32,u32
true,273,423
false,588,272


In [ ]:
# @title
query25 = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("es_oficial") == True)
    .group_by("descripcion_estudio")
    .agg(pl.len().alias("n_alumnos"))
    .with_columns(
        pl.col("n_alumnos").alias("alumnos"),
        pl.col("descripcion_estudio").alias("estudio")
        )
    .select(["estudio","alumnos" ])
)
query25pd = query25.to_pandas()
from google.colab import sheets
sh = sheets.InteractiveSheet(df=query25pd)

https://docs.google.com/spreadsheets/d/1cBGT97H95IlvJp3oS-8uUo2imAolkFoF00BQMUtB6AQ/edit#gid=0


In [ ]:
doc_2024=(
aluf
.filter(pl.col("tipo_estudio").str.to_lowercase().str.contains("doctorado", literal=False))
.filter(pl.col("ano_academico") == 2024)
.collect()
)
media_importe=doc_2024.select(pl.col("importe_academico").mean())
media_creditos=doc_2024.select(pl.col("creditos_matriculados").mean())
precio_medio_doc=(media_importe / media_creditos)
resultado = precio_medio_doc.item()

print(precio_medio_doc)

shape: (1, 1)
┌───────────────────┐
│ importe_academico │
│ ---               │
│ f64               │
╞═══════════════════╡
│ 1176.993007       │
└───────────────────┘


In [ ]:
doc_2024=(
aluf
.filter(pl.col("tipo_estudio").str.to_lowercase().str.contains("doctorado", literal=False))
.filter(pl.col("ano_academico") == 2024)
.collect()
)
query25pd = doc_2024.to_pandas()
from google.colab import sheets
sh = sheets.InteractiveSheet(df=query25pd)

In [ ]:
aluf.schema

<ipython-input-22-364ffdd52150>:1: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  aluf.schema


Schema([('ano_academico', Int64),
        ('descripcion_estudio', String),
        ('pais_nacimiento', String),
        ('dni_pasaporte_alumno', String),
        ('correo_electronico_alumno', String),
        ('correo_electronico2_alumno', String),
        ('fecha_nacimiento', Date),
        ('nip_alumno', Int64),
        ('nombre_alumno', String),
        ('apellido1_alumno', String),
        ('apellido2_alumno', String),
        ('creditos_matriculados', Float64),
        ('sexo_alumno', String),
        ('importe_academico', Float64),
        ('edad_alu', Int32),
        ('es_esi', Boolean),
        ('es_titulopropio', Boolean),
        ('nombre_completo', String),
        ('codigo_estudio', String),
        ('descriptor_agrupado', String),
        ('nuevo_ingreso', Boolean),
        ('ano_acceso', Int64),
        ('apellidos_alumno', String),
        ('descripcion_centro', String),
        ('clase_liquidacion', String),
        ('comunidad_habitual', String),
        ('comunidad_na

In [ ]:
aluf.filter(pl.col("descripcion_plan").str.contains(r"(?i)a distancia")).select(pl.col("descripcion_plan").unique()).to_pandas()

,descripcion_plan
0,Graduado o Graduada en Turismo / Bachelor's De...
1,Máster Universitario en Innovación y Marketing...
2,Máster Universitario en Dirección de Empresas ...
3,Máster Universitario en Derecho Militar - Murc...
4,Graduado o Graduada en Turismo y Dirección de ...
5,Máster Universitario en Ciencias del Matrimoni...
6,Máster Universitario en Responsabilidad Social...
7,"Máster Universitario en Patología, Intervenció..."
8,Máster Universitario en Acceso a las Profesio...
9,Máster Universitario en Dirección de Comunicac...


In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
aluf.filter(
    pl.col("descripcion_plan").str.contains(r"(?i)a distancia"))
    .group_by("descripcion_plan")
    .agg(
    pl.all().first() # Toma el primer valor de cada columna en el grupo
).to_pandas()


,descripcion_plan,ano_academico,descripcion_estudio,pais_nacimiento,dni_pasaporte_alumno,correo_electronico_alumno,correo_electronico2_alumno,fecha_nacimiento,nip_alumno,nombre_alumno,apellido1_alumno,apellido2_alumno,creditos_matriculados,sexo_alumno,importe_academico,edad_alu,es_esi,es_titulopropio,nombre_completo,codigo_estudio,descriptor_agrupado,nuevo_ingreso,ano_acceso,apellidos_alumno,descripcion_centro,clase_liquidacion,comunidad_habitual,comunidad_nacimiento,creditos_reconocidos,curso,curso_mas_alto,edad,estado_condicional,estado_condicional_original,fecha_matricula,importe_administrativo,importe_opcional,localidad_habitual,pais_nacionalidad,plan_codigo,programa_intercambio,provincia_habitual,provincia_nacimiento,rama_conocimiento,tipo_acceso,tipo_beca,tipo_estudio,tipo_matricula,turno,via_acceso,timestamp,es_cartagena_campus,es_online,continente_nacimiento,es_oficial,es_extranjero
0,Máster Universitario en Derecho Militar - Murcia - A Distancia - Castellano,2019,Máster Universitario en Derecho Militar,España,48648354L,maacosta@alu.ucam.edu,manuacosta_hellin@hotmail.com,NaT,90550,Manuel Andrés,Acosta,Hellín,60.0,M,3250.20,NaN,False,False,Manuel Andrés Acosta Hellín,116,MURCIA GENERAL A DISTANCIA,True,2019,Acosta Hellín,FACULTAD DE CIENCIAS JURÍDICAS Y DE LA EMPRESA,Bonificación de Hermanos 5%,Región de Murcia,Región de Murcia,0.0,1,1,25,None,None,2019-10-15,30.00,0.0,San José de la Vega,España,491,N,Murcia,Murcia,Ciencias Sociales y Jurídicas,Acceso a ciclos,None,Máster,Secretaria,No disponible,Acceso a ciclos,2023-10-02 10:58:50.474,False,True,Europa,True,False
1,Máster Universitario en Acceso a las Profesiones de Abogacía y Procura - Murcia - A Distancia - Castellano,2023,Máster Universitario en Acceso a las Profesiones de Abogacía y Procura,España,79058700H,ehernandez10@alu.ucam.edu,elenaujaldon@gmail.com,NaT,68109,Elena,Hernández,Ujaldón,54.0,F,1559.52,NaN,False,False,Elena Hernández Ujaldón,224,MURCIA GENERAL A DISTANCIA,True,2023,Hernández Ujaldón,FACULTAD DE CIENCIAS JURÍDICAS Y DE LA EMPRESA,Sin bonificación,Región de Murcia,Región de Murcia,0.0,1,1,De 26 a 30,None,None,2023-08-09,0.00,0.0,Murcia,España,821,N,Murcia,Murcia,Ciencias Sociales y Jurídicas,Acceso a ciclos,None,Máster,Automatricula,Mañana,Bachillerato Extranjero Homologado,2024-10-17 09:31:27.619,False,True,Europa,True,False
2,Graduado o Graduada en Marketing y Dirección Comercial - Murcia - A Distancia - Castellano,2019,Graduado o Graduada en Marketing y Dirección Comercial,España,48824822P,malbentosa@alu.ucam.edu,miguelalbentosatmi@gmail.com,NaT,70192,Miguel,Albentosa,Vera,60.0,M,4399.80,NaN,False,False,Miguel Albentosa Vera,84,GRADO EN MARKETING Y DIRECCIÓN COMERCIAL MURCIA GENERAL A DISTANCIA CASTELLANO,True,2019,Albentosa Vera,FACULTAD DE CIENCIAS JURÍDICAS Y DE LA EMPRESA,Sin bonificación,Comunidad Valenciana,Comunidad Valenciana,0.0,1,1,22,None,None,2019-08-31,235.00,0.0,Torrevieja,España,706,N,Alicante,Alicante,Ciencias Sociales y Jurídicas,Preinscripción,None,Grado,Automatricula,Mañana,Bachillerato o equivalente (P),2023-10-02 10:58:50.570,False,True,Europa,True,False
3,Graduado o Graduada en Turismo / Bachelor's Degree in Tourism Management - Murcia - A Distancia - Castellano,2019,Graduado o Graduada en Turismo,España,17451997B,pabian@alu.ucam.edu,pabloo9@hotmail.com,NaT,9310,Pablo,Abián,Vicén,3.0,M,137.49,NaN,False,False,Pablo Abián Vicén,97,GRADO EN TURISMO MURCIA GENERAL A DISTANCIA CASTELLANO,False,2012,Abián Vicén,FACULTAD DE CIENCIAS JURÍDICAS Y DE LA EMPRESA,Sin bonificación,Región de Murcia,Aragón,0.0,2,2,De 36 a 45,None,None,2019-10-31,58.88,0.0,Murcia,España,427,N,Murcia,Zaragoza,Ciencias Sociales y Jurídicas,Preinscripción,None,Grado,Secretaria,Mañana,Bachillerato o equivalente (P),2023-10-02 10:58:42.164,False,True,Europa,True,False
4,Máster Universitario en Formación de Profesores de Español como Lengua Extranjera - Murcia - A Distancia - Castellano,2019,Máster Universitario en Formación de Profesores de Español como Lengua E

In [ ]:
# @title
query25 = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("tipo_estudio") == "Doctorado")
    .group_by("descripcion_estudio")
    .agg(pl.len().alias("creditos_matriculados"))
    .with_columns(
        pl.col("n_alumnos").alias("alumnos"),
        pl.col("descripcion_estudio").alias("estudio")
        )
    .select(["estudio","alumnos" ])
)
query25=query25.collect()
query25pd = query25.to_pandas()
from google.colab import sheets
sh = sheets.InteractiveSheet(df=query25pd)

In [ ]:
query29 = aluf.select([
    pl.col("nombre_completo").alias("nombre"),
    pl.col("descripcion_estudio").alias("estudio"),
    pl.col("ano_academico").alias("año_academico"),
    pl.col("correo_electronico_alumno").alias("email_personal"),
    pl.col("correo_electronico2_alumno").alias("email_alternativo"),
])
query29

In [ ]:
print(aluf)

shape: (105_908, 56)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ ano_acade ┆ descripci ┆ pais_naci ┆ dni_pasap ┆ … ┆ es_online ┆ continent ┆ es_oficia ┆ es_extra │
│ mico      ┆ on_estudi ┆ miento    ┆ orte_alum ┆   ┆ ---       ┆ e_nacimie ┆ l         ┆ njero    │
│ ---       ┆ o         ┆ ---       ┆ no        ┆   ┆ bool      ┆ nto       ┆ ---       ┆ ---      │
│ i64       ┆ ---       ┆ str       ┆ ---       ┆   ┆           ┆ ---       ┆ bool      ┆ bool     │
│           ┆ str       ┆           ┆ str       ┆   ┆           ┆ str       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2020      ┆ CURSO     ┆ Irlanda   ┆ PV3279885 ┆ … ┆ null      ┆ Europa    ┆ false     ┆ true     │
│           ┆ ONLINE DE ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ ESPAÑOL   ┆           ┆           ┆   ┆           ┆     

## Mayo 2025

##### 05-05-2025

In [ ]:
# prompt: saca el precio medio de un doctorado de un alumno de 2024

precio_medio_doc = (
    aluf
    .filter(pl.col("tipo_estudio").str.to_lowercase().str.contains("doctorado", literal=False))
    .filter(pl.col("ano_academico") == 2024)
    .select(pl.col("importe_academico").mean() / pl.col("creditos_matriculados").mean())
    )
print(precio_medio_doc.collect())

print(precio_medio_doc.collect()/45)

shape: (1, 1)
┌───────────────────┐
│ importe_academico │
│ ---               │
│ f64               │
╞═══════════════════╡
│ 1176.993007       │
└───────────────────┘
shape: (1, 1)
┌───────────────────┐
│ importe_academico │
│ ---               │
│ f64               │
╞═══════════════════╡
│ 26.1554           │
└───────────────────┘


In [ ]:
nacionalidades_totales=(
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .group_by("pais_nacimiento")
    .agg(pl.count().alias("n_alumnos"))
)
with pl.Config(fmt_str_lengths=120,  # mostrar hasta 1000 caracteres :contentReference[oaicite:5]{index=5}
               tbl_rows=-1):   # sin límite de cantidad de filas
                print(nacionalidades_totales.collect())

shape: (136, 2)
┌────────────────────────────┬───────────┐
│ pais_nacimiento            ┆ n_alumnos │
│ ---                        ┆ ---       │
│ str                        ┆ u32       │
╞════════════════════════════╪═══════════╡
│ Islandia                   ┆ 4         │
│ Venezuela                  ┆ 126       │
│ Emiratos Árabes Unidos     ┆ 12        │
│ Egipto                     ┆ 21        │
│ Serbia                     ┆ 9         │
│ Andorra                    ┆ 2         │
│ Austria                    ┆ 6         │
│ Sri Lanka                  ┆ 1         │
│ Burkina-Faso (Alto Volta)  ┆ 1         │
│                            ┆ 16        │
│ Mongolia                   ┆ 1         │
│ Armenia                    ┆ 5         │
│ India                      ┆ 47        │
│ Georgia                    ┆ 11        │
│ Antillas Holandesas        ┆ 2         │
│ Eslovaquia                 ┆ 6         │
│ Francia                    ┆ 472       │
│ Bangladesh                 ┆ 3      

<ipython-input-21-83427db839eb>:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("n_alumnos"))


##### 06-05-2025

In [ ]:
# prompt: saca el total de alumnos del máster de alto rendimiento del curso 2024/2025

query_master = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(pl.col("descripcion_plan").str.contains("Alto Rendimiento", literal=False))
    .group_by("descripcion_plan")
    .agg(pl.count().alias("n_alumnos"))
)

with pl.Config(fmt_str_lengths=120, tbl_rows=-1):
  print(query_master.collect())


shape: (2, 2)
┌────────────────────────────────────────────────────────────────────────────────┬───────────┐
│ descripcion_plan                                                               ┆ n_alumnos │
│ ---                                                                            ┆ ---       │
│ str                                                                            ┆ u32       │
╞════════════════════════════════════════════════════════════════════════════════╪═══════════╡
│ Máster Universitario en Alto Rendimiento Deportivo: Fuerza y Acondicionamiento ┆ 24        │
│ Físico / Master in High Performance Sport…                                     ┆           │
│ Máster Universitario en Alto Rendimiento Deportivo: Fuerza y Acondicionamiento ┆ 41        │
│ Físico / Master in High Performance Sport…                                     ┆           │
└────────────────────────────────────────────────────────────────────────────────┴───────────┘


<ipython-input-41-924af8885481>:8: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  .agg(pl.count().alias("n_alumnos"))


##### 14-05-2025

In [ ]:
import polars as pl

# Assuming aluf is the LazyFrame
# Filter for online programs and select relevant columns
aluf_online = aluf.filter(pl.col("es_online") == True).select([
    "descripcion_plan",
    "ano_academico",
    "nip_alumno"
])

# Group by program and academic year, count distinct students
aluf_grouped = aluf_online.group_by(["descripcion_plan", "ano_academico"]).agg(
    student_count=pl.col("nip_alumno").n_unique()
)

# Group by program and create columns for each year using conditional aggregation
aluf_aggregated = aluf_grouped.group_by("descripcion_plan").agg(
    count_2023=pl.col("student_count").filter(pl.col("ano_academico") == 2023).sum(),
    count_2024=pl.col("student_count").filter(pl.col("ano_academico") == 2024).sum(),
)

# Replace nulls with 0 and rename columns
result = aluf_aggregated.select([
    pl.col("descripcion_plan"),
    pl.col("count_2023").fill_null(0).alias("2023"),
    pl.col("count_2024").fill_null(0).alias("2024"),
])

# Collect the result to execute the query
result_df = result.collect().to_pandas()

from google.colab import sheets
sheet = sheets.InteractiveSheet(df=result_df)

##### 26-05-2025

In [ ]:
# obtención de datos de presencia de alumnos extranjeros y países representados de los
resultado_pivot = (
    aluf
    .filter(pl.col("ano_academico") == 2024)
    .filter(
        pl.col("descripcion_plan").str.contains("|".join(["Graduado o Graduada en Psicología", "Graduado o Graduada en Ingeniería Informática", "Graduado o Graduada en Marketing y Dirección Comercial"]), literal=False)&
        pl.col("descripcion_plan").str.contains(r"(?i)a distancia")
    )
    .unique(subset=["dni_pasaporte_alumno"])
    .with_columns([
        # Extraer el nombre del máster de la descripción
        pl.when(pl.col("descripcion_plan").str.contains("Graduado o Graduada en Psicología"))
        .then(pl.lit("Psicología"))
        .when(pl.col("descripcion_plan").str.contains("Ingeniería Informática"))
        .then(pl.lit("Informática"))
        .when(pl.col("descripcion_plan").str.contains("Marketing y Dirección Comercial"))
        .then(pl.lit("Marketing"))
        .otherwise(pl.lit("Otro"))
        .alias("grado_nombre"),

        pl.col("es_extranjero").cast(pl.Int32)
    ])
    .group_by("grado_nombre")
    .agg([
        pl.len().alias("total_alumnos"),
        pl.sum("es_extranjero").alias("alumnos_extranjeros"),
        pl.col("pais_nacimiento").n_unique().alias("paises_representados"),
        (pl.sum("es_extranjero") / pl.count() * 100).round(2).alias("porcentaje_extranjeros")
    ])
    .with_columns([
        # Crear las métricas para el pivot
        pl.col("porcentaje_extranjeros").alias("metric_porcentaje"),
        pl.col("paises_representados").alias("metric_paises")
    ])
)

# Crear el DataFrame final pivotado
resultado_final = (
    # Unir porcentajes y países en formato largo
    pl.concat([
        resultado_pivot.select([
            pl.col("grado_nombre"),
            pl.lit("Porcentaje Extranjeros").alias("metrica"),
            pl.col("porcentaje_extranjeros").alias("valor")
        ]),
        resultado_pivot.select([
            pl.col("grado_nombre"),
            pl.lit("Países Representados").alias("metrica"),
            pl.col("paises_representados").cast(pl.Float64).alias("valor")
        ])
    ])
    .pivot(
        index="metrica",
        columns="grado_nombre",
        values="valor"
    )
)


resultado_final

<ipython-input-50-b28a9e70ef72>:28: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  (pl.sum("es_extranjero") / pl.count() * 100).round(2).alias("porcentaje_extranjeros")
<ipython-input-50-b28a9e70ef72>:52: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  .pivot(


metrica,Marketing,Informática,Psicología
str,f64,f64,f64
"""Porcentaje Extranjeros""",9.09,6.97,9.39
"""Países Representados""",5.0,12.0,10.0


In [ ]:
# Query para análisis por sexo
resultado_sexo = (
   aluf
   .filter(pl.col("ano_academico") == 2024)
   .filter(
      pl.col("descripcion_plan").str.contains("|".join(["Graduado o Graduada en Psicología", "Graduado o Graduada en Ingeniería Informática", "Graduado o Graduada en Marketing y Dirección Comercial"]), literal=False)&
      pl.col("descripcion_plan").str.to_lowercase().str.contains("a distancia")
   )
   .unique(subset=["dni_pasaporte_alumno"])
    .with_columns([
    # Extraer el nombre del máster de la descripción
    pl.when(pl.col("descripcion_plan").str.contains("Graduado o Graduada en Psicología"))
    .then(pl.lit("Psicología"))
    .when(pl.col("descripcion_plan").str.contains("Ingeniería Informática"))
    .then(pl.lit("Informática"))
    .when(pl.col("descripcion_plan").str.contains("Marketing y Dirección Comercial"))
    .then(pl.lit("Marketing"))
    .otherwise(pl.lit("Otro"))
    .alias("grado_nombre"),
    ])

   .group_by("grado_nombre")
    .agg([
      pl.len().alias("total_alumnos"),
      (pl.col("sexo_alumno") == "M").sum().alias("num_varones"),
      (pl.col("sexo_alumno") == "F").sum().alias("num_mujeres")
])
)

# Pivot con sexo
pivot_sexo = pl.concat([
   resultado_sexo.select([
       pl.lit("Nº Varones").alias("metrica"),
       pl.col("grado_nombre"),
       pl.col("num_varones").cast(pl.Float64).alias("valor")
   ]),
   resultado_sexo.select([
       pl.lit("Nº Mujeres").alias("metrica"),
       pl.col("grado_nombre"),
       pl.col("num_mujeres").cast(pl.Float64).alias("valor")
   ])
]).pivot(
   index="metrica",
   columns="grado_nombre",
   values="valor"
)

pivot_sexo

<ipython-input-44-9d904908e767>:42: DeprecationWarning: The argument `columns` for `DataFrame.pivot` is deprecated. It has been renamed to `on`.
  ]).pivot(


metrica,Informática,Psicología,Marketing
str,f64,f64,f64
"""Nº Varones""",219.0,65.0,51.0
"""Nº Mujeres""",25.0,148.0,37.0


In [ ]:
cantidad_alumnos = (
  aluf
   .filter(pl.col("ano_academico") == 2024)
   .filter(
      pl.col("descripcion_plan").str.contains("|".join(["Graduado o Graduada en Psicología", "Graduado o Graduada en Ingeniería Informática", "Graduado o Graduada en Marketing y Dirección Comercial"]), literal=False)&
      pl.col("descripcion_plan").str.to_lowercase().str.contains("a distancia")
   )
   .unique(subset=["dni_pasaporte_alumno"])
    .with_columns([
    # Extraer el nombre del máster de la descripción
    pl.when(pl.col("descripcion_plan").str.contains("Graduado o Graduada en Psicología"))
    .then(pl.lit("Psicología"))
    .when(pl.col("descripcion_plan").str.contains("Ingeniería Informática"))
    .then(pl.lit("Informática"))
    .when(pl.col("descripcion_plan").str.contains("Marketing y Dirección Comercial"))
    .then(pl.lit("Marketing"))
    .otherwise(pl.lit("Otro"))
    .alias("grado_nombre"),
    ])

   .group_by("grado_nombre")
    .agg([
      pl.len().alias("total_alumnos"),
      (pl.col("dni_pasaporte_alumno")).len().alias("alu")
])
)
cantidad_alumnos

grado_nombre,total_alumnos,alu
str,u32,u32
"""Marketing""",88,88
"""Informática""",244,244
"""Psicología""",213,213


#### 30-05-2025